In [1]:
# Import packages
# dxpy allows python to interact with the platform storage
# Note: This notebook is using spark since the size of the dataset we're extracting
# (i.e. the number of fields) is too large for a single node instance.
import dxpy
import pandas as pd
import subprocess
import glob
import os
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

In [2]:
output_dir = "/output/"

In [3]:
# Automatically discover dispensed dataset ID
dispensed_dataset = dxpy.find_one_data_object(
    typename="Dataset", name="app*.dataset", folder="/", name_mode="glob"
)
dispensed_dataset_id = dispensed_dataset["id"]

In [4]:
# Get project ID
project_id = dxpy.find_one_project()["id"]

In [5]:
dataset = (":").join([project_id, dispensed_dataset_id])

In [6]:
# Note: This cell can only be run once. Otherwise, you'll need to delete the existing data tables in order to re-run
cmd = ["dx", "extract_dataset", dataset, "-ddd", "--delimiter", ","]
subprocess.check_call(cmd)

0

In [7]:
path = os.getcwd()

In [9]:
data_dict_csv = glob.glob(os.path.join(path, "*.data_dictionary.csv"))[0]
data_dict_df = pd.read_csv(data_dict_csv)
data_dict_df.head()

/tmp/ipykernel_119/118131557.py:2: DtypeWarning: Columns (4,7,8,9,10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_dict_df = pd.read_csv(data_dict_csv)


,entity,name,type,primary_key_type,coding_name,concept,description,folder_path,is_multi_select,is_sparse_coding,linkout,longitudinal_axis_type,referenced_entity_field,relationship,title,units
0,participant,eid,string,global,NaN,NaN,NaN,Participant Information,NaN,NaN,NaN,NaN,NaN,NaN,Participant ID,NaN
1,participant,p3_i0,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 0,seconds
2,participant,p3_i1,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 1,seconds
3,participant,p3_i2,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 2,seconds
4,participant,p3_i3,integer,NaN,NaN,NaN,NaN,Assessment centre > Procedural metrics > Proce...,NaN,NaN,http://biobank.ctsu.ox.ac.uk/crystal/field.cgi...,NaN,NaN,NaN,Verbal interview duration | Instance 3,seconds


In [22]:
print(data_dict_df["entity"].unique())

['participant' 'covid19_result_england' 'covid19_result_scotland'
 'covid19_result_wales' 'gp_clinical' 'gp_scripts' 'gp_registrations'
 'hesin' 'hesin_diag' 'hesin_oper' 'hesin_critical' 'hesin_maternity'
 'hesin_delivery' 'hesin_psych' 'death' 'death_cause' 'olink_instance_0'
 'olink_instance_2' 'olink_instance_3']


In [10]:
field_names = list(
    data_dict_df.loc[data_dict_df["entity"] == "olink_instance_0", "name"].values
)
print(len(field_names))

2924


In [11]:
field_names_str = [f"olink_instance_0.{f}" for f in field_names]
field_names_query = ",".join(field_names_str)

In [12]:
conf = pyspark.SparkConf().set("spark.kryoserializer.buffer.max", "128m")

In [13]:
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
sqlContext = SQLContext(sc)

/cluster/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [14]:
cmd = [
    "dx",
    "extract_dataset",
    dataset,
    "--fields",
    field_names_query,
    "--delimiter",
    ",",
    "--output",
    "extracted_data.sql",
    "--sql",
]
subprocess.check_call(cmd)

0

In [15]:
with open("extracted_data.sql", "r") as file:
    retrieve_sql = ""
    for line in file:
        retrieve_sql += line.strip()

In [16]:
temp_df = spark.sql(retrieve_sql.strip(";"))

In [17]:
pdf = temp_df.toPandas()

In [18]:
print(pdf.shape)
pdf.head()

(53013, 2924)


,olink_instance_0.eid,olink_instance_0.a1bg,olink_instance_0.aamdc,olink_instance_0.aarsd1,olink_instance_0.abca2,olink_instance_0.abhd14b,olink_instance_0.abl1,olink_instance_0.abo,olink_instance_0.abraxas2,olink_instance_0.acaa1,...,olink_instance_0.zfyve19,olink_instance_0.zhx2,olink_instance_0.znf174,olink_instance_0.znf75d,olink_instance_0.znf830,olink_instance_0.znrd2,olink_instance_0.znrf4,olink_instance_0.zp3,olink_instance_0.zp4,olink_instance_0.zpr1
0,1001090,0.1754,0.70995,0.47690,0.13945,0.72730,1.12880,-1.7391,1.3161,1.88660,...,2.0467,0.2292,-0.1017,-0.3249,0.02490,-0.13155,0.6771,2.1311,0.0679,0.4858
1,1001623,0.1047,-0.57265,-0.98630,0.27650,0.47930,-0.58300,NaN,-1.6316,-0.66700,...,-0.8054,-0.1983,0.0506,-0.3820,0.09225,-0.70450,-0.2588,0.8063,-0.5414,0.0976
2,1001945,0.0171,-0.42595,0.08305,0.53980,-0.18125,0.04590,2.5828,0.6365,-0.28125,...,0.6277,0.1305,-0.2988,0.1070,3.49710,0.93995,1.2968,1.5437,NaN,-0.0552
3,1002425,0.4068,0.47565,0.17350,NaN,NaN,0.73650,-3.2560,NaN,0.25040,...,0.9128,0.2516,0.0125,-0.0017,-0.05405,0.22090,0.1690,-4.1724,-0.1351,-1.3381
4,1003685,NaN,NaN,-0.50025,NaN,0.50085,0.26245,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
pdf.to_csv("olink_i0.tsv", sep="\t", index=False, header=True)

In [20]:
%%bash
dx upload olink_i0.tsv -p --path /Output/Olink/ --brief

file-J139qf8JbBGBqfq6B3f7YpQ0


In [26]:
field_names_2 = list(
    data_dict_df.loc[data_dict_df["entity"] == "olink_instance_2", "name"].values
)
print(len(field_names_2))
field_names_str_2 = [f"olink_instance_2.{f}" for f in field_names_2]
field_names_query_2 = ",".join(field_names_str_2)
cmd_2 = [
    "dx",
    "extract_dataset",
    dataset,
    "--fields",
    field_names_query_2,
    "--delimiter",
    ",",
    "--output",
    "extracted_data_2.sql",
    "--sql",
]
subprocess.check_call(cmd_2)
with open("extracted_data_2.sql", "r") as file:
    retrieve_sql = ""
    for line in file:
        retrieve_sql += line.strip()
temp_df_2 = spark.sql(retrieve_sql.strip(";"))
pdf_2 = temp_df_2.toPandas()
print(pdf_2.shape)
pdf_2.head()
pdf_2.to_csv("olink_i2.tsv", sep="\t", index=False, header=True)

1464
(1172, 1464)


In [27]:
%%bash
dx upload olink_i2.tsv -p --path /Output/Olink/ --brief

file-J139zBjJbBG7Zg4F5xXjp4f3


In [28]:
field_names_3 = list(
    data_dict_df.loc[data_dict_df["entity"] == "olink_instance_3", "name"].values
)
print(len(field_names_3))
field_names_str_3 = [f"olink_instance_3.{f}" for f in field_names_3]
field_names_query_3 = ",".join(field_names_str_3)
cmd_3 = [
    "dx",
    "extract_dataset",
    dataset,
    "--fields",
    field_names_query_3,
    "--delimiter",
    ",",
    "--output",
    "extracted_data_3.sql",
    "--sql",
]
subprocess.check_call(cmd_3)
with open("extracted_data_3.sql", "r") as file:
    retrieve_sql = ""
    for line in file:
        retrieve_sql += line.strip()
temp_df_3 = spark.sql(retrieve_sql.strip(";"))
pdf_3 = temp_df_3.toPandas()
print(pdf_3.shape)
pdf_3.head()
pdf_3.to_csv("olink_i3.tsv", sep="\t", index=False, header=True)

1464
(1123, 1464)


In [29]:
%%bash
dx upload olink_i3.tsv -p --path /Output/Olink/ --brief

file-J13B088JbBG6ZBpYFJ1Fp7vx
